In [ ]:
import os

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
!pip install python-docx
import docx
!pip install PyPDF2
import PyPDF2

!pip install transformers datasets

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)
# # note: the path may be different for others.
# os.chdir('/content/drive/MyDrive/DAP Group 6: Fin Scanners/statements')

In [ ]:
!ls

Applications                    monetary20220615a1_summary.docx
Desktop                         monetary20220727a1.pdf
Developer                       monetary20220727a1_summary.docx
Documents                       monetary20220921a1.pdf
Downloads                       monetary20220921a1_summary.docx
Library                         monetary20221102a1.pdf
Movies                          monetary20221102a1_summary.docx
Music                           monetary20221214a1.pdf
Pictures                        monetary20221214a1_summary.docx
Postman                         news_summary.csv
Public                          nltk_data
Sites                           open
Summary 20220126a1.docx         opt
Summary 20220504a1.docx         pandas
Summary 20220615a1.docx         refsum20220126.docx
Summary 20220727a1.docx         refsum20220504.docx
Summary 20220921a1.docx         refsum20220615.docx
Summary 20221102a1.docx         refsum20220727.docx
Summary 20221214a1.docx         refsum20220921.do

# create a function to preprocess the statement to be fed into the model

In [ ]:
def clean_monetary_pdf(monetary_pdf, by="para"):
  # takes in 2 parameters: first is the original statement;
  # second parameter is "para" (for split by paragraphs) or  "sen" (for split by first sentences)

  # opening the pdf
  monetary_pdf = open(monetary_pdf, "rb")
  monetary_pdf = PyPDF2.PdfReader(monetary_pdf)

  # chunking all the text in the pdf together
  text = ""
  for page in monetary_pdf.pages:
    text+=page.extract_text()
    if "-0-" in page:
      break

  # create a header variable which we will need to take out if present
  header = ""
  start = text.find("For release")
  end = text.find("22")
  header = text[start:end+2]

  text = text.split("\n")

  # split by paragraph
  para_summary_input = []
  current_para = ""

  for i,sentence in enumerate(text):
    if "Voting" in sentence:
      break
    elif "-2-" in sentence or "(more)" in sentence:
      continue
    # make sure there's no headers in the sentence
    sentence = sentence.replace(header, "")
    # if first word is a " ", that means it's a start of a new paragraph! (not the same for every text!)
    # if the sentence starts with a capital letter and the prevoious letter is a " " then it's also a new para! (idk how that works)
    # push the current para into the summary and reset current_para
    # print(sentence)
    if sentence[0] == " " or (sentence[0].isupper() and text[i-1][-1] == " "):
      if current_para.strip():
        para_summary_input.append(current_para.strip())
      current_para = sentence
    else:
        current_para += sentence
  # add in the leftovers
  if current_para:
    para_summary_input.append(current_para)

  if by == "para":
    return para_summary_input

  # split and get the first sentences
  if by == "sen":
    first_n_summary = ""
    for para in para_summary_input:      
      first_n_summary += para.split(". ")[0] + ". "
    
    return first_n_summary

(sort of\) works for everything except for the statements in Jun, Jul and Sep

In [ ]:
# by_para_mon202212 = clean_monetary_pdf("monetary20221214a1.pdf", by="para")
# by_sen_mon202212 = clean_monetary_pdf("monetary20221214a1.pdf", by="sen")

In [ ]:
by_para_mon202212 = ['Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures.', 'Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks.', 'The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time.  In determining the pace of future increases in the target range, the Committee will take into account the cumulative tightening of monetary policy, the lags  with which monetary policy affects economic activity and inflation, and economic and financial developments.  In addition, the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities, as d escribed in the Plans for Reducing the Size of the Federal Reserve’s Balance Sheet that were issued in May.  The Committee is strongly committed to returning inflation to its 2 percent objective.', 'In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook.  The Committee would be prepared to adjust the stance of monetary policy as appropriate if ris ks emerge that could impede the attainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments.   ']
by_sen_mon202212 = "Recent indicators point to modest growth in spending and production. Russia’s war against Ukraine is causing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. "

In [ ]:
# by_para_mon202211 = clean_monetary_pdf("monetary20221102a1.pdf", by="para")
# by_sen_mon202211 = clean_monetary_pdf("monetary20221102a1.pdf", by="sen")

In [ ]:
by_para_mon202211 = ['Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures .', 'Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related  events are creating additional upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks.', 'The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent ov er the longer run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 3 -3/4 to 4 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stance of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time .  In determining the pace of future increases in the target range, the Committee will take into account the cumulative tightening of monetary policy, the lags with which monetary policy affects economic activity and inflation, and economic and financial developments.  In addition, the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities, as described in the Plans for Reducing the Size of the Federal Reserve’s Balance Sheet that were issued in May.  The Committee is strongly committed to returning inflation to its 2 percent objective.', ' In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook.  The Committee would be prepared to adjust the stance of monetary policy as appropriate if ris ks emerge that could impede the attainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments.   ']
by_sen_mon202211 = "Recent indicators point to modest growth in spending and production. Russia’s war against Ukraine is causing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent ov er the longer run.  In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. "

In [ ]:
# by_para_mon202205 = clean_monetary_pdf("monetary20220504a1.pdf", by="para")
# by_sen_mon202205 = clean_monetary_pdf("monetary20220504a1.pdf", by="sen")

In [ ]:
by_para_mon202205 = ['Although overall economic activity edged down in the first quarter, household spending and business fixed investment remained strong.  Job gains have been robust  in recent months, and the unemployment rate has declined substantially.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher energy prices, and broader price pressures.', 'The invasion of Ukraine by Russia is c ausing tremendous human and economic hardship.  The implications for the U.S. economy are highly uncertain.  The invasion and related events are creating  additional upward pressure on inflation and are likely to weigh on economic activity.  In addition, CO VID- related lockdowns in China are likely to exacerbate supply chain disruptions.  The Committee is highly attentive to inflation risks.', 'The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run.  With appropriate firming in the stance of monetary policy, the Committee expects inflation to return to its 2  percent objective and the labor market to remain strong.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 3/4 to 1 percent and anticipates that ongoing increases in the target range will be appropriate.  In addition, the Committee decided  to begin reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities on June 1, as described in the Plans for Reducing the Size of the Federal Reserve’s Balance Sheet that were issued in conjunction with this statement.', 'In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic', '  outlook.  The Committee would be prepared to adjust the stance of monetary policy as appropriate if risks emerge that could impede the attainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments.  ']
by_sen_mon202205 = "Although overall economic activity edged down in the first quarter, household spending and business fixed investment remained strong. The invasion of Ukraine by Russia is causing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic.   outlook. "

In [ ]:
# by_para_mon202201 = clean_monetary_pdf("monetary20220126a1.pdf", by="para")
# by_sen_mon202201 = clean_monetary_pdf("monetary20220126a1.pdf", by="sen")

In [ ]:
by_para_mon202201 = ['Indicators of economic activity and employment have continued to strengthen. The sectors most adversely affected by the pandemic have improved in recent months but are being affected by the recent sharp rise in COVID-19 cases.  Job gains have been solid in recent months, and the unemployment rate has declined substantially.  Supply and demand imbalances related to the pandemic and the reopening of the economy have continued to contribute to elevated levels of inflation.  Overall financial conditions remain accommodative, in part reflecting policy measures to support the economy and the flow of credit to U.S. households and businesses.', 'The path of the economy continues to depend on the course of the virus.  Progress on vaccinations and an easing of supply cons traints are expected to support continued gains in economic activity and employment as well as a reduction in inflation.  Risks to the economic outlook remain, including from new va riants of the virus.', 'The Committee seeks to achieve maximum em ployment and inflation at the rate of 2 percent over the longer run.  In s upport of these goals, the Committee decided to keep the target range for the federal funds rate  at 0 to 1/4 percent.  With inflation well above 2 percent and a strong labor market , the Committee expects it will soon be appropriate to raise the target range for the federal funds rate.   The Committee decided to continue to redu ce the monthly pace of its net asset purchases, bringing them to an end in early March.  Beginning in February, the Committee will increase its holdings of Treasury securities by at least $20 billion per month and of agency mortgage ‑backed securities by at l east $10 billion per month.  The Federal Reserve’s ongoing purchases and  holdings of securities will continue to foster smooth market functioning and accommodative financial conditions, thereby supporting the flow of credit to households and businesses.', 'In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of  incoming information for the economic outlook.  The Committee would be prepared to  adjust the stance of monetary policy as appropriate if risks emer ge that could impede the at tainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readi ngs on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments. ']
by_sen_mon202201 = "Indicators of economic activity and empl oyment have continued to strengthen. The sectors most adversely affected by th e pandemic have improved in recent months but are being affected by the recent sharp rise in COVID-19 cases. The path of the economy continues to depe nd on the course of the virus. The Committee seeks to achieve maximum em ployment and inflation at the rate of 2 percent over the longer run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of  incoming information for the economic outlook. "

In [ ]:
by_para_mon = [by_para_mon202212, by_para_mon202211, by_para_mon202205, by_para_mon202201]
by_sen_mon = [by_sen_mon202212, by_sen_mon202211, by_sen_mon202205, by_sen_mon202201]

# code to extract our human-summarisation from the docx into the model

In [ ]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return ' '.join(fullText)

In [ ]:
  # given the way i write the for-loop later on, the order should stay like that

human202212 = getText("refsum20221214.docx")
human202211 = getText("refsum20221102.docx")
human202205 = getText("refsum20220504.docx")
human202201 = getText("refsum20220126.docx")
tarasha202212 = getText("Summary 20221214a1.docx")
tarasha202211 = getText("Summary 20221102a1.docx")
tarasha202205 = getText("Summary 20220504a1.docx")
tarasha202201 = getText("Summary 20220126a1.docx")
arjun202212 = getText("monetary20220126a1_summary.docx")
arjun202211 = getText("monetary20221102a1_summary.docx")
arjun202205 = getText("monetary20220504a1_summary.docx")
arjun202201 = getText("monetary20220126a1_summary.docx")

by_human = [human202212, human202211, human202205, human202201]
by_tarasha = [tarasha202212, tarasha202211, tarasha202205, tarasha202201]
by_arjun = [arjun202212, arjun202211, arjun202205, arjun202201]

# Getting GPT2 model (from PyPI) to evaluate text that is split by paragraph

In [ ]:
!pip install bert-extractive-summarizer
from summarizer import Summarizer,TransformerSummarizer

In [ ]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

In [ ]:
print(GPT2_model.model)

functools.partial(<summarizer.transformer_embeddings.bert_embedding.BertEmbedding object at 0x7fd83219d670>, hidden=-2, reduce_option='mean', hidden_concat=False)


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

def calc_rouge_scores(model_summary, reference_summary):
    return rouge.get_scores(model_summary, reference_summary)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from nltk.corpus import stopwords
from nltk import download

download('stopwords')
stop_words = stopwords.words('english')

def preprocess(sentence):
  a = []
  for sen in sentence:
    for word in sen.split(" "):
      if word.lower() not in stop_words:
        a.append(word.lower())
  return a

def preprocess2(sentence):
  return [w for w in sentence.lower().split() if w not in stop_words]

def preprocess_to_str(sentence):
  sentence = sentence.lower().split()
  result = ""
  for i in sentence:
    if i not in stop_words:
      result += (i + " ")
  return result

[nltk_data] Downloading package stopwords to /Users/yj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# avg score from 1st summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_human_rouge = {}
gpt2_generated_summaries = []
N = len(by_human)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  gpt2_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    gpt2_candidate.append(GPT2_model(para, min_length=50))
  gpt2_cand = ' '.join(gpt2_candidate)
  gpt2_generated_summaries.append(gpt2_cand)
  print(gpt2_cand)

  result = calc_rouge_scores(preprocess_to_str(gpt2_cand), preprocess_to_str(by_human[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_human_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_human_rouge)

Recent indicators point to modest growth in spending and production. Job gains have been robust in recent months, and the unemployment rate has remained low. Russia’s war against Ukraine is causing tremendous human and economic hardship. The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation 

In [ ]:
# avg score from 2nd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_tarasha_rouge = {}

N = len(by_tarasha)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  gpt2_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    gpt2_candidate.append(GPT2_model(para, min_length=50))
  gpt2_cand = ' '.join(gpt2_candidate)
  print(gpt2_cand)
    
  result = calc_rouge_scores(preprocess_to_str(gpt2_cand), preprocess_to_str(by_tarasha[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_tarasha_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_tarasha_rouge)

Recent indicators point to modest growth in spending and production. Job gains have been robust in recent months, and the unemployment rate has remained low. Russia’s war against Ukraine is causing tremendous human and economic hardship. The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation 

In [ ]:
# avg score from 3rd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_arjun_rouge = {}

N = len(by_arjun)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  gpt2_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    gpt2_candidate.append(GPT2_model(para, min_length=50))
  gpt2_cand = ' '.join(gpt2_candidate)
  print(gpt2_cand)
    
  result = calc_rouge_scores(preprocess_to_str(gpt2_cand), preprocess_to_str(by_arjun[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_arjun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_arjun_rouge)

Recent indicators point to modest growth in spending and production. Job gains have been robust in recent months, and the unemployment rate has remained low. Russia’s war against Ukraine is causing tremendous human and economic hardship. The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation 

In [ ]:
# function to get the combined avg of all 3 results

def avg_rouge_results(all_results):
  N = len(all_results)
  ans = {'rouge-1':{'r': 0, 'p': 0, 'f': 0}, 'rouge-2':{'r': 0, 'p': 0, 'f': 0}, 'rouge-l':{'r': 0, 'p': 0, 'f': 0}}

  # to get the sum of the score
  for i in range(len(all_results)):
    for k, (r, p, f) in all_results[i].items():
      ans[k][r] += all_results[i][k][r]
      ans[k][p] += all_results[i][k][p]
      ans[k][f] += all_results[i][k][f]
  
  # to get the avg score
  for k, (r, p, f) in ans.items():
    ans[k][r] /= N
    ans[k][p] /= N
    ans[k][f] /= N
  return ans

In [ ]:
# ## Combined avg of all 3 summaries
all_gpt2_rouge = [avg_gpt2_by_human_rouge, avg_gpt2_by_tarasha_rouge, avg_gpt2_by_arjun_rouge]
print("*** Average ROUGE scores ***")
print(avg_rouge_results(all_gpt2_rouge))

*** Average ROUGE scores ***
{'rouge-1': {'r': 0.45326256733511694, 'p': 0.27478018469397775, 'f': 0.3284276848502943}, 'rouge-2': {'r': 0.23477702520398155, 'p': 0.12616493419469352, 'f': 0.15709192683890155}, 'rouge-l': {'r': 0.44239551953193845, 'p': 0.2655068032223205, 'f': 0.31875139825217813}}


## Based on WMD

In [ ]:
import gensim.downloader as api
wmdmodel = api.load('word2vec-google-news-300')
!pip install pyemd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
avg_dist = 0

for i in range(N):
  reference_summary1 = preprocess2(by_human[i])
  reference_summary2 = preprocess2(by_tarasha[i])
  reference_summary3 = preprocess2(by_arjun[i])
  model_summary = preprocess2(gpt2_generated_summaries[i])
  print(f"*** WMD for the {i+1}th summary ***")
  distance1 = wmdmodel.wmdistance(model_summary, reference_summary1)
  print('distance = %.4f' % distance1)
  distance2 = wmdmodel.wmdistance(model_summary, reference_summary2)
  print('distance = %.4f' % distance2)
  distance3 = wmdmodel.wmdistance(model_summary, reference_summary3)
  print('distance = %.4f' % distance3)
  dist = (distance1 + distance2 + distance3) / 3
  avg_dist += dist
  print('average distance = %.4f' % dist + "\n")
print("*** Average WMD from all 4 summaries *** ")
print(avg_dist/4)

*** WMD for the 1th summary ***
distance = 0.8960
distance = 0.8232
distance = 1.0143
average distance = 0.9111

*** WMD for the 2th summary ***
distance = 0.7297
distance = 0.7497
distance = 0.8776
average distance = 0.7856

*** WMD for the 3th summary ***
distance = 0.7626
distance = 0.8860
distance = 0.6460
average distance = 0.7649

*** WMD for the 4th summary ***
distance = 0.8746
distance = 0.6750
distance = 0.9377
average distance = 0.8291

*** Average WMD from all 4 summaries *** 
0.8226913831081987


In [ ]:
print(reference_summary1)
print(reference_summary2)
print(reference_summary3)
print(model_summary)

['economy', 'employment', 'continues', 'strengthen,', 'inflation', 'high.', 'committee', 'committed', 'achieving', 'maximum', 'employment', 'returning', 'inflation', 'rate', '2', 'percent', 'committee', 'keep', 'target', 'range', 'federal', 'funds', 'rate', '0', '1/4', 'percent.', 'committee', 'increase', 'holdings', 'securities.', 'due', 'inflation', '2', 'percent', 'strong', 'labour', 'market,', 'committee', 'expects', 'raise', 'target', 'range', 'federal', 'funds', 'rate.', 'committee', 'reduce', 'monthly', 'pace', 'net', 'asset', 'purchases,', 'stop', 'early', 'march.']
['paragraph', 'paragraph:', 'improvement', 'sectors', 'affected', 'pandemic.', 'unemployment', 'declined.', 'imbalance', 'supply', 'demand', 'fuelled', 'inflation.', 'economy', 'still', 'dependent', 'pandemic', 'situation.', 'committee’s', 'goal', 'increase', 'employment', 'get', 'inflation', '2%.', 'achieved,', 'federal', 'funds', 'rate', 'increased.', 'committee', 'continue', 'monitor', 'adjust', 'monetary', 'poli

# Getting GPT2 model from Huggingface to evaluate text that is split by paragraph

In [ ]:
!pip install transformers
import re
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch.optim as optim

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelWithLMHead.from_pretrained("gpt2")
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
# tokenizer.encode(" TL;DR ")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
tokenizer.encode(" TL;DR ")

[24811, 26, 7707, 220]

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

def calc_rouge_scores(model_summary, reference_summary):
    return rouge.get_scores(model_summary, reference_summary)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from nltk.corpus import stopwords
from nltk import download

download('stopwords')
stop_words = stopwords.words('english')

def preprocess(sentence):
  a = []
  for sen in sentence:
    for word in sen.split(" "):
      if word.lower() not in stop_words:
        a.append(word.lower())
  return a

def preprocess2(sentence):
  return [w for w in sentence.lower().split() if w not in stop_words]

def preprocess_to_str(sentence):
  sentence = sentence.lower().split()
  result = ""
  for i in sentence:
    if i not in stop_words:
      result += (i + " ")
  return result

[nltk_data] Downloading package stopwords to /Users/yj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 512, 200, 512 works
# 1024, 200, 512 works

# max_length in tokenizer should be > mex_length in generate
N = len(by_para_mon[0])
list_of_summaries = ["mon202212", "mon202211", "mon202205", "mon202201"]
new_gpt2_summary = []
by_para_mon2 = []
for speech in by_para_mon:
  by_para_mon2.append(" ".join(speech))
for i in range(N):
  print(f"*** Monetary policy for {list_of_summaries[i]} ***")
  print("Original text: "+ by_para_mon2[i])

  GPT_summary=""
  input_ids = tokenizer.encode(by_para_mon2[i] +'TL;DR', return_tensors='pt', truncation=True)
  summary_ids=model.generate(
    input_ids,
    do_sample=True, 
    max_length = 1024,
    min_length = 512,
    top_k=50, 
    top_p=0.90, 
    num_return_sequences=4,
    pad_token_id=tokenizer.eos_token_id,
  )
  GPT_summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
  GPT_summary = GPT_summary.split("TL;DR")[1]
  
  new_gpt2_summary.append(GPT_summary)
  print("Summarised text: "+ GPT_summary + "\n")

*** Monetary policy for mon202212 ***
Original text: Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures. Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary p

## Using rouge

In [ ]:
# avg score from 1st summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_new_gpt2_by_human_rouge = {}

N = len(by_human)

for i in range(N):  
  for generated_summary in new_gpt2_summary:
    result = calc_rouge_scores(preprocess_to_str(generated_summary), preprocess_to_str(by_human[i]))
    rouge1r += result[0]['rouge-1']['r']
    rouge1p += result[0]['rouge-1']['p']
    rouge1f += result[0]['rouge-1']['f']

    rouge2r += result[0]['rouge-2']['r']
    rouge2p += result[0]['rouge-2']['p']
    rouge2f += result[0]['rouge-2']['f']

    rougelr += result[0]['rouge-l']['r']
    rougelp += result[0]['rouge-l']['p']
    rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_human_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_human_rouge)

{'rouge-1': {'r': 1.0558536082731746, 'p': 0.8893739725658231, 'f': 0.9508610732975666}, 'rouge-2': {'r': 0.29625281599965647, 'p': 0.2334564981463363, 'f': 0.25813981762104304}, 'rouge-l': {'r': 1.0017847775585143, 'p': 0.8457231789150293, 'f': 0.903200028008865}}


In [ ]:
# avg score from 2nd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_new_gpt2_by_tarasha_rouge = {}

N = len(by_tarasha)

for i in range(N):  
  for generated_summary in new_gpt2_summary:
    result = calc_rouge_scores(preprocess_to_str(generated_summary), preprocess_to_str(by_tarasha[i]))
    rouge1r += result[0]['rouge-1']['r']
    rouge1p += result[0]['rouge-1']['p']
    rouge1f += result[0]['rouge-1']['f']

    rouge2r += result[0]['rouge-2']['r']
    rouge2p += result[0]['rouge-2']['p']
    rouge2f += result[0]['rouge-2']['f']

    rougelr += result[0]['rouge-l']['r']
    rougelp += result[0]['rouge-l']['p']
    rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_tarasha_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_tarasha_rouge)

{'rouge-1': {'r': 0.7544675429202254, 'p': 0.743626539466947, 'f': 0.7011019582166418}, 'rouge-2': {'r': 0.14621141567452828, 'p': 0.14572496615541997, 'f': 0.1322534093773565}, 'rouge-l': {'r': 0.7152674255483376, 'p': 0.7090137979356994, 'f': 0.6660394454954252}}


In [ ]:
# avg score from 3rd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_arjun_rouge = {}

N = len(by_arjun)

for i in range(N):  
  for generated_summay in new_gpt2_summary: 
    result = calc_rouge_scores(preprocess_to_str(generated_summary), preprocess_to_str(by_arjun[i]))
    rouge1r += result[0]['rouge-1']['r']
    rouge1p += result[0]['rouge-1']['p']
    rouge1f += result[0]['rouge-1']['f']

    rouge2r += result[0]['rouge-2']['r']
    rouge2p += result[0]['rouge-2']['p']
    rouge2f += result[0]['rouge-2']['f']

    rougelr += result[0]['rouge-l']['r']
    rougelp += result[0]['rouge-l']['p']
    rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_arjun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_arjun_rouge)

{'rouge-1': {'r': 0.8215102974828375, 'p': 0.9677419354838711, 'f': 0.8881513243067138}, 'rouge-2': {'r': 0.23103609143729592, 'p': 0.2676056338028169, 'f': 0.24779781717084245}, 'rouge-l': {'r': 0.6872616323417237, 'p': 0.8064516129032256, 'f': 0.7416749441761683}}


In [ ]:
all_gpt2_rouge = [avg_gpt2_by_human_rouge, avg_gpt2_by_tarasha_rouge, avg_gpt2_by_arjun_rouge]
# SCORE TO BEAT: {'rouge-1': {'r': 0.44843257775721174, 'p': 0.26704870321249635, 'f': 0.32233283893970527}, 'rouge-2': {'r': 0.23409951842891383, 'p': 0.12404817840146716, 'f': 0.15570658372256482}, 'rouge-l': {'r': 0.43756552995403325, 'p': 0.2579605069260242, 'f': 0.3127341419547412}}
print(avg_rouge_results(all_gpt2_rouge))

{'rouge-1': {'r': 0.8772771495587458, 'p': 0.8669141491722137, 'f': 0.8467047852736407}, 'rouge-2': {'r': 0.2245001077038269, 'p': 0.21559569936819103, 'f': 0.212730348056414}, 'rouge-l': {'r': 0.8014379451495252, 'p': 0.7870628632513181, 'f': 0.7703048058934862}}


## Using wmd


In [ ]:
avg_dist = 0

for i in range(N):
  reference_summary1 = preprocess2(by_human[i])
  reference_summary2 = preprocess2(by_tarasha[i])
  reference_summary3 = preprocess2(by_arjun[i])
  model_summary = preprocess2(new_gpt2_summary[i])
  print(f"*** WMD for the {i+1}th summary ***")
  distance1 = wmdmodel.wmdistance(model_summary, reference_summary1)
  print('distance = %.4f' % distance1)
  distance2 = wmdmodel.wmdistance(model_summary, reference_summary2)
  print('distance = %.4f' % distance2)
  distance3 = wmdmodel.wmdistance(model_summary, reference_summary3)
  print('distance = %.4f' % distance3)
  dist = (distance1 + distance2 + distance3) / 3
  avg_dist += dist
  print('average distance = %.4f' % dist + "\n")
print("*** Average WMD from all 4 summaries *** ")
print(avg_dist/N)

*** WMD for the 1th summary ***
distance = 0.9105
distance = 1.0540
distance = 1.0481
average distance = 1.0042

*** WMD for the 2th summary ***
distance = 0.9183
distance = 1.0539
distance = 0.8701
average distance = 0.9474

*** WMD for the 3th summary ***
distance = 0.9351
distance = 0.9360
distance = 1.0350
average distance = 0.9687

*** WMD for the 4th summary ***
distance = 0.8771
distance = 0.9140
distance = 0.8868
average distance = 0.8926

*** Average WMD from all 4 summaries *** 
0.9532546757460377


# Finetuning

In [ ]:
!pip install transformers
import re
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch.optim as optim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
# news_articles_data = pd.read_csv("/content/drive/MyDrive/DAP Group 6: Fin Scanners/news_summary.csv", encoding = "ISO-8859-1")
news_articles_data = pd.read_csv("news_summary.csv", encoding="ISO-8859-1")
news_articles_data.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
news_articles_data.rename(columns = {"text":"summary", "ctext":"full_text"}, inplace = True)
news_articles_data.head()

,author,date,headlines,read_more,summary,full_text
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
len(news_articles_data)

4514

## Preparing data

In [ ]:
news_articles_data["joined_text"] = news_articles_data["full_text"] + " TL;DR " + news_articles_data["summary"]
news_articles_data.head()

,author,date,headlines,read_more,summary,full_text,joined_text
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo...","From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
news_articles_data["joined_text"] = news_articles_data["joined_text"].astype(str)

In [ ]:
# find avg length for inputs
avg_length = sum([len(article.split()) for article in news_articles_data["joined_text"]])/len(news_articles_data["joined_text"])
avg_length = round(avg_length)

print(avg_length)

max_length = 500

392


In [ ]:
class ArticlesFinetuningDataset(Dataset):  
    def __init__(self, tokenizer, reviews, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.reviews = reviews
        self.result = []

        for review in self.reviews:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(review + self.eos)
            
            # Padding/truncating the encoded sequence to max_len 
            padded = self.pad_truncate(tokenized)            

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

In [ ]:
# # if you want to use the og one, use this code 
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelWithLMHead.from_pretrained("gpt2")
# model = model.to(device)

In [ ]:
# if you want to use the fine-tuned one, use this code 
tokenizer = AutoTokenizer.from_pretrained("yijiyap/finscan_gpt2_test")
model = AutoModelWithLMHead.from_pretrained("yijiyap/finscan_gpt2_test")
model = model.to(device)

/Users/yj/opt/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
tokenizer.encode(" TL;DR ")

[24811, 26, 7707, 220]

In [ ]:
extra_length = len(tokenizer.encode(" TL;DR "))

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [ ]:
# generate dataset
dataset = ArticlesFinetuningDataset(tokenizer, news_articles_data["joined_text"], 500)

Token indices sequence length is longer than the specified maximum sequence length for this model (1221 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, drop_last=True)

## Train

In [ ]:
def train(model, optimizer, dl, epochs):    
    for epoch in range(epochs):
        for idx, batch in enumerate(dl):
             with torch.set_grad_enabled(True):
                optimizer.zero_grad()
                batch = batch.to(device)
                output = model(batch, labels=batch)
                loss = output[0]
                loss.backward()
                optimizer.step()
                if idx % 50 == 0:
                    print("loss: %f, %d"%(loss, idx))

In [ ]:
# Prevent all layers other than the last 2 from changing
N = len(model.transformer.h)
for i, m in enumerate(model.transformer.h):
  if i >= N-2:
    for parameter in m.parameters():
      parameter.requires_grad = True
  else:
    for parameter in m.parameters():
      parameter.requires_grad = False

In [ ]:

# # run to fine-tune model
# train(model=model, optimizer=optimizer, dl=dataloader, epochs=1)


## Newly summarised model

In [ ]:
# change the max length maybe idk
N = len(by_para_mon[0])
list_of_summaries = ["mon202212", "mon202211", "mon202205", "mon202201"]
newer_gpt2_summary = []
by_para_mon2 = []
for speech in by_para_mon:
  by_para_mon2.append(" ".join(speech))
for i in range(N):
  print(f"*** Monetary policy for {list_of_summaries[i]} ***")
  print("Original text: "+ by_para_mon2[i])

  GPT_summary=""
  input_ids = tokenizer.encode(by_para_mon2[i] +'TL;DR', return_tensors='pt', truncation=True)
  summary_ids=model.generate(
    input_ids,
    do_sample=True, 
    max_length=1500, 
    min_length = 800,
    top_k=50, 
    top_p=0.90, 
    num_return_sequences=4,
    pad_token_id=tokenizer.eos_token_id
  )
  GPT_summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
  GPT_summary = GPT_summary.split("TL;DR")[1]
  
  newer_gpt2_summary.append(GPT_summary)
  print("Summarised text: "+ GPT_summary + "\n")

*** Monetary policy for mon202212 ***
Original text: Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures. Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary p

## Saving the model


In [ ]:
# model.push_to_hub("finscan_gpt2_test")

In [ ]:
# tokenizer.push_to_hub("finscan_gpt2_test")

# Evaluating new summary


## Using rouge

In [ ]:
# avg score from 1st summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_new_gpt2_by_human_rouge = {}

N = len(by_human)

for i in range(N):  
  for generated_summary in newer_gpt2_summary:
    result = calc_rouge_scores(preprocess_to_str(generated_summary), preprocess_to_str(by_human[i]))
    rouge1r += result[0]['rouge-1']['r']
    rouge1p += result[0]['rouge-1']['p']
    rouge1f += result[0]['rouge-1']['f']

    rouge2r += result[0]['rouge-2']['r']
    rouge2p += result[0]['rouge-2']['p']
    rouge2f += result[0]['rouge-2']['f']

    rougelr += result[0]['rouge-l']['r']
    rougelp += result[0]['rouge-l']['p']
    rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_human_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_human_rouge)

{'rouge-1': {'r': 1.0132109759448458, 'p': 0.8314564564564565, 'f': 0.8677905533680633}, 'rouge-2': {'r': 0.26792453467185506, 'p': 0.19951923076923078, 'f': 0.214640626915637}, 'rouge-l': {'r': 0.9766370869033049, 'p': 0.8115326865326866, 'f': 0.8420541248544068}}


In [ ]:
# avg score from 2nd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_new_gpt2_by_tarasha_rouge = {}

N = len(by_tarasha)

for i in range(N):  
  for generated_summary in newer_gpt2_summary:
    result = calc_rouge_scores(preprocess_to_str(generated_summary), preprocess_to_str(by_tarasha[i]))
    rouge1r += result[0]['rouge-1']['r']
    rouge1p += result[0]['rouge-1']['p']
    rouge1f += result[0]['rouge-1']['f']

    rouge2r += result[0]['rouge-2']['r']
    rouge2p += result[0]['rouge-2']['p']
    rouge2f += result[0]['rouge-2']['f']

    rougelr += result[0]['rouge-l']['r']
    rougelp += result[0]['rouge-l']['p']
    rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_tarasha_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_tarasha_rouge)

{'rouge-1': {'r': 0.9330889720021658, 'p': 0.910833910833911, 'f': 0.8324177037003151}, 'rouge-2': {'r': 0.19636118033006958, 'p': 0.15661057692307692, 'f': 0.14884461955000486}, 'rouge-l': {'r': 0.8580274432220094, 'p': 0.8248440748440747, 'f': 0.7614200304878089}}


In [ ]:
# avg score from 3rd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_arjun_rouge = {}

N = len(by_arjun)

for i in range(N):  
  for generated_summay in newer_gpt2_summary: 
    result = calc_rouge_scores(preprocess_to_str(generated_summary), preprocess_to_str(by_arjun[i]))
    rouge1r += result[0]['rouge-1']['r']
    rouge1p += result[0]['rouge-1']['p']
    rouge1f += result[0]['rouge-1']['f']

    rouge2r += result[0]['rouge-2']['r']
    rouge2p += result[0]['rouge-2']['p']
    rouge2f += result[0]['rouge-2']['f']

    rougelr += result[0]['rouge-l']['r']
    rougelp += result[0]['rouge-l']['p']
    rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_arjun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_arjun_rouge)

{'rouge-1': {'r': 0.2620137299771167, 'p': 0.730769230769231, 'f': 0.3855521000347513}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.2620137299771167, 'p': 0.730769230769231, 'f': 0.3855521000347513}}


In [ ]:
all_gpt2_rouge = [avg_gpt2_by_human_rouge, avg_gpt2_by_tarasha_rouge, avg_gpt2_by_arjun_rouge]
# SCORE TO BEAT: {'rouge-1': {'r': 0.44843257775721174, 'p': 0.26704870321249635, 'f': 0.32233283893970527}, 'rouge-2': {'r': 0.23409951842891383, 'p': 0.12404817840146716, 'f': 0.15570658372256482}, 'rouge-l': {'r': 0.43756552995403325, 'p': 0.2579605069260242, 'f': 0.3127341419547412}}
print(avg_rouge_results(all_gpt2_rouge))

{'rouge-1': {'r': 0.7361045593080427, 'p': 0.8243531993531995, 'f': 0.69525345236771}, 'rouge-2': {'r': 0.15476190500064155, 'p': 0.1187099358974359, 'f': 0.12116174882188062}, 'rouge-l': {'r': 0.698892753367477, 'p': 0.7890486640486641, 'f': 0.6630087517923223}}


## Using wmd


In [ ]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords')
stop_words = stopwords.words('english')

def preprocess(sentence):
  a = []
  for sen in sentence:
    for word in sen.split(" "):
      if word.lower() not in stop_words:
        a.append(word.lower())
  return a

def preprocess2(sentence):
  return [w for w in sentence.lower().split() if w not in stop_words]

def preprocess_to_str(sentence):
  sentence = sentence.lower().split()
  result = ""
  for i in sentence:
    if i not in stop_words:
      result += (i + " ")
  return result

avg_dist = 0
for i in range(N):
  reference_summary1 = preprocess2(by_human[i])
  reference_summary2 = preprocess2(by_tarasha[i])
  reference_summary3 = preprocess2(by_arjun[i])
  model_summary = preprocess2(newer_gpt2_summary[i])
  print(f"*** WMD for the {i+1}th summary ***")
  distance1 = wmdmodel.wmdistance(model_summary, reference_summary1)
  print('distance = %.4f' % distance1)
  distance2 = wmdmodel.wmdistance(model_summary, reference_summary2)
  print('distance = %.4f' % distance2)
  distance3 = wmdmodel.wmdistance(model_summary, reference_summary3)
  print('distance = %.4f' % distance3)
  dist = (distance1 + distance2 + distance3) / 3
  avg_dist += dist
  print('average distance = %.4f' % dist + "\n")
print("*** Average WMD from all 4 summaries *** ")
print(avg_dist/N)

*** WMD for the 1th summary ***
distance = 0.9573
distance = 0.9569
distance = 0.9006
average distance = 0.9383

*** WMD for the 2th summary ***
distance = 0.8385
distance = 1.0016
distance = 1.0329
average distance = 0.9577

*** WMD for the 3th summary ***
distance = 0.8750
distance = 1.0220
distance = 0.9462
average distance = 0.9477

*** WMD for the 4th summary ***
distance = 1.0498
distance = 0.9819
distance = 1.1364
average distance = 1.0561

*** Average WMD from all 4 summaries *** 
0.9749342021736258


In [ ]:
print(preprocess2(new_gpt2_summary[i]))

['committee', 'international', 'financial', 'management', 'considered', 'changing', 'monetary', 'policy', 'united', 'states', 'mix', 'add', 'accommodative', 'monetary', 'policy', 'improve', 'financial', 'international', 'conditions.', 'committee', 'aware', 'risk', 'increasing', 'rate', 'inflation', 'response', 'pandemic', 'present', 'adopted', 'accommodative', 'policies.']


In [ ]:
print(reference_summary3)

['us', 'economy', 'showed', 'signs', 'recovery', 'strong', 'indicators', 'good', 'employment', 'despite', 'fact', 'got', 'severely', 'affected', 'covid-19', 'cases.', 'but,', 'supply-demand', 'imbalances', 'due', 'pandemic', 'rise', 'new', 'virus', 'variants.', 'federal', 'reserve', 'kept', 'federal', 'funds', 'rate', 'target', 'range', '0%', '¼%', 'expects', 'raise', 'soon', 'due', 'inflation', 'well', '2%', 'strong', 'labor', 'market.', 'furthermore,', 'committee', 'decided', 'continue', 'reduce', 'monthly', 'pace', 'net', 'asset', 'purchases.', 'also', 'aim', 'increase', 'holdings', 'treasury', 'securities', 'least', '$20', 'billion', 'month', 'agency', 'mortgage‐backed', 'securities', 'least', '$10', 'billion', 'per', 'month.', 'purchase', 'holdings', 'aim', 'continue', 'smoothen', 'market', 'functioning,', 'thereby', 'supporting', 'flow', 'credit', 'households', 'businesses.']
